> Forecasting without Sentiments

**Table of contents**<a id='toc0_'></a>    
1. [Setup](#toc1_)    
2. [Model](#toc2_)    
3. [Functions](#toc3_)    
3.1. [Prepare Data](#toc3_1_)    
3.2. [Train & Evaluate](#toc3_2_)    
3.3. [Load Data](#toc3_3_)    
3.4. [Predict](#toc3_4_)    
3.5. [Get Combinations](#toc3_5_)    
4. [Hyperparameter tuning](#toc4_)    
5. [User Call](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Setup](#toc0_)

In [181]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from pprint import pprint
from prettytable import PrettyTable
import yfinance as yf
import datetime as dt
from itertools import combinations
import pickle

# 2. <a id='toc2_'></a>[Model](#toc0_)

In [24]:
class MultivariateLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MultivariateLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [157]:
class DynamicLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_layers, output_size):
        super(DynamicLSTM, self).__init__()
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.LSTM(input_size, hidden_sizes[0], num_layers=1, batch_first=True))
        for i in range(1, len(hidden_sizes)):
            self.hidden_layers.append(nn.LSTM(hidden_sizes[i-1], hidden_sizes[i], num_layers=1, batch_first=True))
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        for layer in self.hidden_layers:
            x, _ = layer(x)
        x = self.output_layer(x[:, -1, :])  # Take the output from the last time step
        return x

# 3. <a id='toc3_'></a>[Functions](#toc0_)

## 3.1. <a id='toc3_1_'></a>[Prepare Data](#toc0_)

In [25]:
def prepare_data_multivariate(df, choosen_stock, startdate, enddate, features, look_back, predict_type='year'):
    # Choose specific stock
    data = df[df["Stock"] == choosen_stock]

    # Test split
    if predict_type=='year':
        test_data = data[data["Date"].dt.year == 2019]
    elif predict_type=='month':
        test_data = data[(data["Date"].dt.year == 2019) & (data["Date"].dt.month.isin([1]))]
    elif predict_type=='days':
        test_data = data[data["Date"].dt.year == 2019][0:20] 
    else: # Specific
        test_data = data[(data["Date"] >= dt.datetime(2023, 11, 12)) & (data["Date"] <= enddate)]
        
    # Train split
    train_data = data[(data["Date"] >= startdate) & (data["Date"] <= dt.datetime(2023, 11, 11))]
    
    # Feature selection and engineering
    train_data = train_data[features + ["Date"]].values
    test_data = test_data[features + ["Date"]].values
    
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_data[:, :-1] = scaler.fit_transform(train_data[:, :-1])
    test_data[:, :-1] = scaler.transform(test_data[:, :-1])
    
    # Create sequences for LSTM input
    def create_sequences(dataset, look_back=1):
        X, Y, dates = [], [], []
        for i in range(len(dataset) - look_back):
            X.append(dataset[i:(i + look_back), :-1])
            Y.append(dataset[i + look_back, 0])
            dates.append(dataset[i + look_back, -1])  # Assuming the last column is 'Date'
        return np.array(X), np.array(Y), np.array(dates)
    train_X, train_Y, train_dates = create_sequences(train_data, look_back)
    test_X, test_Y, test_dates = create_sequences(test_data, look_back)

    # Convert data to PyTorch tensors
    train_X = torch.Tensor(train_X.astype(np.float32))
    train_Y = torch.Tensor(train_Y)
    test_X = torch.Tensor(test_X.astype(np.float32))
    test_Y = torch.Tensor(test_Y)
    
    return train_X, train_Y, train_dates, test_X, test_Y, test_dates, scaler, test_data

## 3.2. <a id='toc3_2_'></a>[Train & Evaluate](#toc0_)

In [163]:
def train_evaluate_lstm_multivariate(input_size, hidden_sizes, num_layers, output_size, learning_rate, num_epochs, train_X, train_Y, test_X, test_Y, scaler, test_data, visualize=True):
    # Initialize the model
    # model = MultivariateLSTMModel(input_size, hidden_sizes, num_layers, output_size)
    model = DynamicLSTM(input_size, hidden_sizes, num_layers, output_size)
    # print(model)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
    # Training the model
    train_losses = []
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        outputs = model(train_X)
        optimizer.zero_grad()
        loss = criterion(outputs.view(-1), train_Y)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    # Calculate predictions
    model.eval()
    train_predict = model(train_X).view(-1).cpu().detach().numpy()
    test_predict = model(test_X).view(-1).cpu().detach().numpy()
    
    # Compute RMSE & MAPE
    train_rmse = mean_squared_error(train_Y, train_predict, squared=False)
    test_rmse = mean_squared_error(test_Y, test_predict, squared=False)
    train_mape = mean_absolute_percentage_error(train_Y, train_predict)
    test_mape = mean_absolute_percentage_error(test_Y, test_predict)
    
    # Inverse Scaling
    # --> 1.test_predict
    test_data1 = test_data[:, 1:-1]
    # Ensure the second array has the same number of rows as the first array
    test_data1 = test_data1[:test_predict.reshape(-1, 1).shape[0], :]
    # Append the arrays
    test_data1 = np.hstack((test_predict.reshape(-1, 1), test_data1)) 
    test_predict_inverse = scaler.inverse_transform(test_data1)[:,0]
    
    # --> 2.test_Y
    test_data2 = test_data[:, :-1]
    test_data2 = test_data2[:test_predict.reshape(-1, 1).shape[0], :]
    test_Y_inverse = scaler.inverse_transform(test_data2)[:,0]
    
    # Visualize test and predictions
    if visualize == True:
        plt.figure(figsize=(10, 5))
        plt.plot(test_dates, test_Y_inverse, label='True', linewidth=2)
        plt.plot(test_dates, test_predict_inverse, label='Predicted', linewidth=2)
        plt.title("Test vs. Predicted Prices")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.show()
    
    return model, loss, train_rmse, test_rmse, train_mape, test_mape, test_Y_inverse, test_predict_inverse 

## 3.3. <a id='toc3_3_'></a>[Load Data](#toc0_)

In [51]:
def load_data(choosen_stock):    
    yf.pdr_override() # Override pandas datareader with yfinance
    y_symbols = [choosen_stock]
    
    # State the dates
    startdate = dt.datetime(2018, 1, 1) # start date
    enddate = dt.datetime(2023, 11, 30) # end date # +1
    
    # Retrieve historical stock price data for the specified symbols and date range
    df = yf.download(y_symbols, start=startdate, end=enddate) 
    df = df.reset_index() # Reset the index to make 'Date' a regular column
    df['Stock'] = choosen_stock # add 'Stock' column
    df = df[['Date', 'Stock', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']] # Reorder the columns
    df['Date'] = pd.to_datetime(df['Date'])
    
    return df, startdate, enddate

## 3.4. <a id='toc3_4_'></a>[Predict](#toc0_)

In [172]:
def predict(choosen_stock):
    # Load the data
    data, startdate, enddate = load_data(choosen_stock)

    # Append the last row to the DataFrame
    new_row = [{'Date':pd.to_datetime('2023-11-27T00:00:00.000000000'), 'Stock': choosen_stock , 'Adj Close': 0.0, 'Close': 0.0 , 'High': 0.0,'Low': 0.0, 'Open':0.0, 'Volume':0.0}]
    data = pd.concat([data, pd.DataFrame(new_row)], ignore_index=True)

    # 1. Hyperparameters
    look_back = 5 # No. of Lags to consider
    predict_type = 'Predict' # Predict type ['Year', 'Month', 'Days','Predict']
    features = ['Close', 'High', 'Low', 'Open', 'Volume']
    predict_type = 'Predict'                            # Predict type ['Year', 'Month', 'Days','Predict']
    hidden_sizes = [64, 64]                   # Adjust the hidden_size values as needed
    num_layers = 2                       # [1, 2, 3]
    learning_rate = 0.001         # [0.005, 0.01, 0.02]  
    num_epochs = 100                                    # [50, 100, 200] 

    # Prepare the data
    train_X, train_Y, train_dates, test_X, test_Y, test_dates, scaler, test_data = prepare_data_multivariate(data, choosen_stock, startdate, enddate, features=features, look_back=look_back, predict_type=predict_type )

    # 2. Hyperparameters
    input_size = 5  # Number of input features (High, Low, Open, Close, Volume)
    output_size = 1  # Number of output features (Close price)
    num_epochs = 100

    # Create the model
    model, loss, train_rmse, test_rmse, train_mape, test_mape, test_Y_inverse, test_predict_inverse  = train_evaluate_lstm_multivariate(input_size, hidden_sizes, num_layers, output_size, learning_rate, num_epochs, train_X, train_Y, test_X, test_Y, scaler, test_data, visualize=False)
    
    # Formatting the prices to a desired decimal form
    formatted_test_Y = ["{:.4f}".format(price) for price in test_Y_inverse.flatten()]
    formatted_test_predict = ["{:.4f}".format(price) for price in test_predict_inverse.flatten()]
    formatted_dates = [test_dates.strftime('%Y-%m-%d') for test_dates in test_dates]

    return formatted_test_Y, formatted_test_predict, formatted_dates

## 3.5. <a id='toc3_5_'></a>[Get Combinations](#toc0_)

In [173]:
def get_combinations(options, num_layers):
    combinations_list = []
    for r in range(1, num_layers+1):  # To get combinations of 1, 2, ... num_layers
        combinations_list.extend(combinations(options * r, r))
    combinations_list = [list(comb) for comb in combinations_list]
    # Filter combinations based on length
    filtered_combinations = [comb for comb in combinations_list if len(comb) == num_layers]
    return filtered_combinations

# 4. <a id='toc4_'></a>[Hyperparameter tuning](#toc0_)

In [176]:
## Tuning

# Dictionary to store the best models for each ticker
all_best_models = {}

for ticker in ['GOOG', 'MSFT', 'META', 'AMZN', 'NFLX', 'TSLA', 'NVDA']:
    print("\n" + ticker)
    
    # Hyperparameters
    choosen_stock = "GOOG"                              # ['GOOG', 'MSFT', 'META', 'AMZN', 'NFLX', 'TSLA', 'NVDA']
    look_back_values = [5]                              # No. of Lags to consider [3, 5, 10, 20]
    predict_type = 'Predict'                            # Predict type ['Year', 'Month', 'Days','Predict']
    hidden_size_options = [64, 128]                      # Adjust the hidden_size values as needed
    num_layers_values = [1, 2]                       # [1, 2, 3]
    learning_rate_values = [0.001, 0.01, 0.005]         # [0.005, 0.01, 0.02]  
    num_epochs = 100                                    # [50, 100, 200] 
    top_k = 1                                           # Get the top k performing models

    # ?
    years_to_include = [2015, 2016, 2017, 2018, 2019]
    train_days_values = [-1, 60, 100, 365, 365*2]


    # Features
    features=['High', 'Low', 'Open', 'Close', 'Volume']
    input_size = 5  # Number of input features (High, Low, Open, Close, Volume)
    output_size = 1  # Number of output features (Close price)


    # Load the data
    df, startdate, enddate = load_data(choosen_stock) 

    # Store best_models
    best_models = []

    # Hyperparameter tuning loop

    for num_layers in num_layers_values:
        hidden_size_values = get_combinations(hidden_size_options, num_layers)
        # print(hidden_size_values)
        for hidden_size in hidden_size_values:
            # print(hidden_size)
            for look_back in look_back_values:
                for learning_rate in learning_rate_values:
                    # print(f"\nHyperparameters: look_back={look_back}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}")
                    # Prepare the data
                    train_X, train_Y, train_dates, test_X, test_Y, test_dates, scaler, test_data = prepare_data_multivariate(df, choosen_stock, startdate, enddate, features=features, look_back=look_back, predict_type=predict_type)
                    model, loss, train_rmse, test_rmse, train_mape, test_mape, test_Y_inverse, test_predict_inverse = train_evaluate_lstm_multivariate(input_size, hidden_size, num_layers, output_size, learning_rate, num_epochs, train_X, train_Y, test_X, test_Y, scaler, test_data, visualize=False)
                    best_models.append({
                        "model" : model,
                        "look_back": look_back,
                        "learning_rate": learning_rate,
                        "hidden_size" : hidden_size,
                        "num_layers" : num_layers,
                        "test_rmse": test_rmse,
                        "test_mape": test_mape,
                    })

    # Sort the models by RMSE in ascending order
    best_models.sort(key=lambda x: (x["test_rmse"], x["test_mape"]))

    # Store the top-k performing models for the current ticker
    all_best_models[ticker] = best_models[:top_k]

# Print the best models for each ticker
for ticker, models in all_best_models.items():
    print(f"\nTop {top_k} Performing Model for {ticker}:")
    pprint(models)    


GOOG
[*********************100%%**********************]  1 of 1 completed

MSFT
[*********************100%%**********************]  1 of 1 completed

META
[*********************100%%**********************]  1 of 1 completed

AMZN
[*********************100%%**********************]  1 of 1 completed

NFLX
[*********************100%%**********************]  1 of 1 completed

TSLA
[*********************100%%**********************]  1 of 1 completed

NVDA
[*********************100%%**********************]  1 of 1 completed

Top 1 Performing Model for GOOG:
[{'hidden_size': [64, 128],
  'learning_rate': 0.001,
  'look_back': 5,
  'model': DynamicLSTM(
  (hidden_layers): ModuleList(
    (0): LSTM(5, 64, batch_first=True)
    (1): LSTM(64, 128, batch_first=True)
  )
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
),
  'num_layers': 2,
  'test_mape': 0.0072738146,
  'test_rmse': 0.0077783647}]

Top 1 Performing Model for MSFT:
[{'hidden_size': [128, 128],
  'learning_rate'

In [179]:
# print the models

# Assuming all_best_models is a dictionary with tickers as keys and a list of best models as values
table = PrettyTable()
table.field_names = ["Ticker", "Model", "Look Back", "Learning Rate", "Hidden Size", "Num Layers", "Test RMSE", "Test MAPE"]

for ticker, models in all_best_models.items():
    for idx, model_info in enumerate(models, start=1):
        table.add_row([
            ticker,
            idx,
            model_info["look_back"],
            model_info["learning_rate"],
            model_info["hidden_size"],
            model_info["num_layers"],
            round(model_info["test_rmse"], 4),
            round(model_info["test_mape"], 4),
        ])

# Print the combined table
print("Top Performing Models:")
print(table)

Top Performing Models:
+--------+-------+-----------+---------------+-------------+------------+-----------+-----------+
| Ticker | Model | Look Back | Learning Rate | Hidden Size | Num Layers | Test RMSE | Test MAPE |
+--------+-------+-----------+---------------+-------------+------------+-----------+-----------+
|  GOOG  |   1   |     5     |     0.001     |  [64, 128]  |     2      |   0.0078  |   0.0073  |
|  MSFT  |   1   |     5     |     0.001     |  [128, 128] |     2      |   0.0078  |   0.0063  |
|  META  |   1   |     5     |     0.001     |  [128, 128] |     2      |   0.0077  |   0.0072  |
|  AMZN  |   1   |     5     |     0.001     |     [64]    |     1      |   0.0084  |   0.0087  |
|  NFLX  |   1   |     5     |     0.001     |   [64, 64]  |     2      |   0.008   |   0.0064  |
|  TSLA  |   1   |     5     |     0.001     |  [128, 128] |     2      |   0.0077  |   0.0075  |
|  NVDA  |   1   |     5     |     0.001     |  [64, 128]  |     2      |   0.0079  |   0.0083 

In [184]:
# Save model weights
for ticker, models in all_best_models.items():
    for idx, model_info in enumerate(models, start=1):
        # Save the model weights to a pickle file
        model_filename = f"{ticker}_model.pkl"
        with open("Saved Models/"+model_filename, 'wb') as file:
            pickle.dump(model_info["model"].state_dict(), file)
        print(f"Model weights for {ticker} saved to {model_filename}")

Model weights for GOOG saved to GOOG_model.pkl
Model weights for MSFT saved to MSFT_model.pkl
Model weights for META saved to META_model.pkl
Model weights for AMZN saved to AMZN_model.pkl
Model weights for NFLX saved to NFLX_model.pkl
Model weights for TSLA saved to TSLA_model.pkl
Model weights for NVDA saved to NVDA_model.pkl


# 5. <a id='toc5_'></a>[User Call](#toc0_)

In [185]:
# Take user input for choosen_stock
# choosen_stock = input("Enter the stock symbol ('GOOG', 'MSFT', 'META', 'AMZN', 'NFLX', 'TSLA', 'NVDA'): ")
choosen_stock = 'GOOG'

# Predict
actual_prices, predicted_prices, formatted_dates = predict(choosen_stock)

# Display
# Remove the first value and shift up the remaining values
actual_prices = actual_prices[1:] + ['']
table = PrettyTable()
table.field_names = ["Date", "Actual Price", "Predicted Price"]
for date, actual_price, predicted_price in zip(formatted_dates, actual_prices, predicted_prices):
    # Add a separator line before the last row
    if date == formatted_dates[-1]:
        table.add_row(["-" * 10, "-" * 12, "-" * 16])
    table.add_row([date, round(float(actual_price), 2) if actual_price != '' else 'TBA', round(float(predicted_price), 2)])
print(table)

[*********************100%%**********************]  1 of 1 completed
+------------+--------------+------------------+
|    Date    | Actual Price | Predicted Price  |
+------------+--------------+------------------+
| 2023-11-20 |    135.43    |      136.71      |
| 2023-11-21 |    136.38    |      137.44      |
| 2023-11-22 |    138.7     |      137.85      |
| 2023-11-24 |    136.94    |      138.32      |
| ---------- | ------------ | ---------------- |
| 2023-11-27 |     TBA      |      138.57      |
+------------+--------------+------------------+
